<a href="https://colab.research.google.com/github/noora20FH/PCVK/blob/main/leaf_classification_PCVK_finalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Noora Aulia Hidayat

TI-3I

2041720046

https://github.com/noora20FH/PCVK/blob/main/leaf_classification_PCVK_finalProject.ipynb 

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

MessageError: ignored